# 📊 Análise de Batalhas - Clash Royale + MongoDB

In [18]:
from pymongo import MongoClient
from datetime import datetime
from config import MONGO_URI, DB_NAME

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
battles = db["battles"]


## 🔍 Consulta 1 — % de Vitórias/Derrotas com uma Carta Específica

In [19]:
# Parâmetros da consulta
carta = "Rocket"
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)

# Pipeline
pipeline1 = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim},
            "$or": [
                {"team.cards.name": carta},
                {"opponent.cards.name": carta}
            ]
        }
    },
    {
        "$group": {
            "_id": "$is_winner",
            "total": {"$sum": 1}
        }
    }
]

# Execução
result1 = list(battles.aggregate(pipeline1))

# Processar resultados
vitorias = sum(doc["total"] for doc in result1 if doc["_id"] == True)
derrotas = sum(doc["total"] for doc in result1 if doc["_id"] == False)
total = vitorias + derrotas

print(f"Carta: {carta}")
print(f"Período: {inicio.date()} até {fim.date()}")

if total == 0:
    print("Nenhuma batalha encontrada com essa carta no intervalo fornecido.")
else:
    print(f"Total de batalhas: {total}")
    print(f"Vitórias: {vitorias} ({(vitorias/total)*100:.2f}%)")
    print(f"Derrotas: {derrotas} ({(derrotas/total)*100:.2f}%)")


Carta: Rocket
Período: 2025-04-01 até 2025-04-30
Total de batalhas: 33
Vitórias: 15 (45.45%)
Derrotas: 18 (54.55%)


## 🔍 Consulta 2 — Decks com mais de X% de vitórias

In [ ]:
# Parâmetros
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)
min_porcentagem_vitoria = 60  # Exemplo: 60%

# Conexão
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
battles = db["battles"]

# Pipeline
pipeline2 = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim}
        }
    },
    {
        "$project": {
            "deck": {
                "$sortArray": {
                    "input": {
                        "$map": {
                            "input": {
                                "$ifNull": [
                                    { "$getField": {
                                        "field": "cards",
                                        "input": { "$arrayElemAt": ["$team", 0] }
                                    }},
                                    []
                                ]
                            },
                            "as": "card",
                            "in": "$$card.name"
                        }
                    },
                    "sortBy": 1
                }
            },
            "is_winner": 1
        }
    },
    {
        "$group": {
            "_id": "$deck",
            "total": {"$sum": 1},
            "vitorias": {
                "$sum": {
                    "$cond": [{"$eq": ["$is_winner", True]}, 1, 0]
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "deck": "$_id",
            "total": 1,
            "vitorias": 1,
            "porcentagem": {
                "$multiply": [
                    {"$divide": ["$vitorias", "$total"]},
                    100
                ]
            }
        }
    },
    {
        "$match": {
            "porcentagem": {"$gt": min_porcentagem_vitoria}
        }
    },
    {
        "$sort": {"porcentagem": -1}
    }
]


# Execução  
result2 = list(battles.aggregate(pipeline2))

# Exibição
print(f"Decks com mais de {min_porcentagem_vitoria}% de vitórias entre {inicio.date()} e {fim.date()}")
for deck in result2:
    print(f"Deck: {deck['deck']}")
    print(f"Vitórias: {deck['vitorias']} / {deck['total']} ({deck['porcentagem']:.2f}%)\n")


## 🔍 Consulta 3 — Derrotas com combo de cartas (X1, X2, ...) em um intervalo de tempo

In [21]:
# Parâmetros da consulta
combo = ["Knight", "Zap"]
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)

# Conexão
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
battles = db["battles"]

# Pipeline
pipeline3 = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim},
            "is_winner": False  # derrota do player (você)
        }
    },
    {
        "$project": {
            "cartas": {
                "$map": {
                    "input": {
                        "$ifNull": [
                            { "$getField": {
                                "field": "cards",
                                "input": { "$arrayElemAt": ["$team", 0] }
                            }},
                            []
                        ]
                    },
                    "as": "card",
                    "in": "$$card.name"
                }
            }
        }
    },
    {
        "$match": {
            "cartas": {
                "$all": combo  # verificar se TODAS as cartas do combo estão presentes
            }
        }
    },
    {
        "$count": "total_derrotas"
    }
]

# Execução
result3 = list(battles.aggregate(pipeline3))

# Exibição
print(f"Combo: {combo}")
print(f"Período: {inicio.date()} até {fim.date()}")
if result3:
    print(f"Total de derrotas com esse combo: {result3[0]['total_derrotas']}")
else:
    print("Nenhuma derrota encontrada com esse combo no intervalo.")

Combo: ['Knight', 'Zap']
Período: 2025-04-01 até 2025-04-30
Total de derrotas com esse combo: 19
